# Deteccão de COVID-19 em imagens de raio-X

O objetivo deste exercício é treinar e avaliar um classificador capaz de detectar a presença de COVID-19 em imagens de raio-X.

As métricas a serem reportadas são F1 e Equal Error Rate (ERR) e seu respectivo threshold.

Nota 1: uma das dificuldades deste exercicio será processar as imagens, que tem 1100 x 1100 pixels.

Nota 2: Não podemos utilizar outros datasets de COVID para treinamento, pois o nosso dataset foi construído a partir de vários outros disponíveis na internet. Mas podemos usar outros datasets que não são de COVID para ajudar no transfer learning.

Dica: Utilize o [pyeer](https://pypi.org/project/pyeer/) para plotar o EER.


## Inicializando o Neptune

In [ ]:
! pip install neptune-client==0.9.1

In [ ]:
import neptune.new as neptune

# Insira seu api_token para logar os resultados do treino na sua conta do Neptune.
# Como obter seu API token do Neptune:
# https://docs.neptune.ai/administration/security-and-privacy/how-to-find-and-set-neptune-api-token

run = neptune.init(project='rodrigonogueira/Aula-5-Exemplo', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhYzhlOTIxYy1lMGNhLTRiY2QtYTdjYi1jNWMyN2YxNzVhMTQifQ==')

https://app.neptune.ai/rodrigonogueira/Aula-5-Exemplo/e/AUL1-128


## Importação das bibliotecas

In [ ]:
!pip install pytorch_lightning

In [ ]:
from collections import OrderedDict
import h5py
import numpy as np

import torch
from torch.utils.data import DataLoader
from torch import nn

import torchvision
from torchvision import transforms

import pytorch_lightning
from pytorch_lightning import LightningModule, Trainer

pytorch_lightning.utilities.seed.seed_everything(seed=123)

Error occurred during asynchronous operation processing: Invalid point for string series: monitoring/stdout : Text longer than 1000 characters was truncated
Error occurred during asynchronous operation processing: Invalid point for string series: monitoring/stdout : Text longer than 1000 characters was truncated
Error occurred during asynchronous operation processing: Invalid point for string series: monitoring/stdout : Text longer than 1000 characters was truncated
Global seed set to 123


123

## Dataset e dataloader

### Carregamento, criação dataset e do dataloader

In [ ]:
# O download demora.
!gsutil -m cp -n gs://neuralresearcher_data/unicamp/ia376i_2021s1/data/xray_covid/* .

Skipping existing item: file://./test.hdf5
Skipping existing item: file://./train.hdf5
Skipping existing item: file://./valid.hdf5


Aqui criamos um dataset sem aumento de dados

In [ ]:
size_final = (224, 224)

transform_train = transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize(size=size_final)])

transform_valid = transform_train

In [ ]:
class Hdf5_Dataset(torch.utils.data.Dataset):

    def __init__(self, path, transform):

        h5file = h5py.File(path, 'r')
        # Hack para deixar mais rapido o carregamento das imagens.
        self.images = h5file['data'][:1000]
        self.labels = h5file['label'][:1000]
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx].astype(np.int64)
        img = self.images[idx].astype(np.uint8)
        img = self.transform(img)
        img = img.tile(3, 1, 1)
      
        return img, label

In [ ]:
train_dataset = Hdf5_Dataset('./train.hdf5', transform=transform_train)
val_dataset = Hdf5_Dataset('./valid.hdf5', transform=transform_valid)
test_dataset = Hdf5_Dataset('./test.hdf5', transform=transform_valid)

403
404
442


In [ ]:
batch_size = 50

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print('Número de minibatches de trenamento:', len(train_dataloader))
print('Número de minibatches de validação:', len(val_dataloader))
print('Número de minibatches de teste:', len(test_dataloader))

x_train, y_train = next(iter(train_dataloader))
print("\nDimensões dos dados de um minibatch:", x_train.size())
print("Valores mínimo e máximo dos pixels: ", torch.min(x_train), torch.max(x_train))
print("Tipo dos dados das imagens:         ", type(x_train))
print("Tipo das classes das imagens:       ", type(y_train))

Número de minibatches de trenamento: 20
Número de minibatches de validação: 20
Número de minibatches de teste: 20

Dimensões dos dados de um minibatch: torch.Size([50, 3, 224, 224])
Valores mínimo e máximo dos pixels:  tensor(0.) tensor(1.)
Tipo dos dados das imagens:          <class 'torch.Tensor'>
Tipo das classes das imagens:        <class 'torch.Tensor'>
